In [1]:
import re

import pandas as pd
from vietadminunits import parse_address, get_data
from vietadminunits.utils import to_key, to_alphanumeric
import pickle

In [2]:
df = pd.DataFrame(get_data())

In [3]:
df['long_province_alphanumeric'] = df['long_province'].apply(to_alphanumeric)
df['long_district_alphanumeric'] = df['long_district'].apply(to_alphanumeric)
df['long_ward_alphanumeric'] = df['long_ward'].fillna('').apply(to_alphanumeric)

df['province_alphanumeric'] = df['province'].apply(to_alphanumeric)
df['short_district_alphanumeric'] = df['short_district'].apply(to_alphanumeric)
df['short_ward_alphanumeric'] = df['short_ward'].fillna('').apply(to_alphanumeric)

In [4]:
for ward in ['long_ward', 'ward']:
    for district in ['long_district', 'district']:
        for province in ['long_province', 'province']:
            df[f"address_{ward}_{province}_{district}"] = df[ward].fillna('') + df[province] + df[district]
            df[f"address_comma_{ward}_{province}_{district}"] = df[ward].fillna('') + ',' + df[province] + ',' + df[district]

In [5]:
df['province_key'] = df['province'].apply(to_key, args=(1,))
df['district_key'] = df['district'].apply(to_key, args=(2,))
df['ward_key'] = df['ward'].apply(to_key, args=(3,))

In [6]:
address_cols = [col for col in df.columns if 'address' in col]

## Find wards that are the same keyword with provinces

In [13]:
wrong_provinces = []



for address_col in address_cols:
    for row in df.itertuples():
        address = getattr(row, address_col)
        unit = parse_address(address, level=1)
        data = {
            'address': address,
            'province': row.province,
            'province_key': row.province_key,
            'district_key': row.district_key,
            'ward_key': row.ward_key,
            'wrong_province_key': unit.province_key,
        }
        
 
            
        if unit.province != row.province:
            print(address)
            wrong_provinces.append(data)

Xã Tịnh HàTỉnh Quảng NgãiHuyện Sơn Tịnh
Thị trấn Cái Tàu HạĐồng ThápHuyện Châu Thành
Xã An HiệpĐồng ThápHuyện Châu Thành
Xã An NhơnĐồng ThápHuyện Châu Thành
Xã Tân Nhuận ĐôngĐồng ThápHuyện Châu Thành
Xã Tân BìnhĐồng ThápHuyện Châu Thành
Xã Tân Phú TrungĐồng ThápHuyện Châu Thành
Xã Phú LongĐồng ThápHuyện Châu Thành
Xã An Phú ThuậnĐồng ThápHuyện Châu Thành
Xã Phú HựuĐồng ThápHuyện Châu Thành
Xã An KhánhĐồng ThápHuyện Châu Thành
Xã Tân PhúĐồng ThápHuyện Châu Thành
Xã Hòa TânĐồng ThápHuyện Châu Thành
Xã Tịnh HàTỉnh Quảng NgãiSơn Tịnh
Xã Sơn HàTỉnh Phú YênSơn Hòa
Xã Xuân MinhHà GiangQuang Bình
Xã Tiên NguyênHà GiangQuang Bình
Xã Tân NamHà GiangQuang Bình
Xã Bản RịaHà GiangQuang Bình
Xã Yên ThànhHà GiangQuang Bình
Thị trấn Yên BìnhHà GiangQuang Bình
Xã Tân TrịnhHà GiangQuang Bình
Xã Tân BắcHà GiangQuang Bình
Xã Bằng LangHà GiangQuang Bình
Xã Yên HàHà GiangQuang Bình
Xã Hương SơnHà GiangQuang Bình
Xã Xuân GiangHà GiangQuang Bình
Xã Nà KhươngHà GiangQuang Bình
Xã Tiên YênHà GiangQuang Bình
Xã 

In [14]:
df_wrong_provinces = pd.DataFrame(wrong_provinces)

In [15]:
df_wrong_provinces.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   address             432 non-null    object
 1   province            432 non-null    object
 2   province_key        432 non-null    object
 3   district_key        432 non-null    object
 4   ward_key            432 non-null    object
 5   wrong_province_key  432 non-null    object
dtypes: object(6)
memory usage: 20.4+ KB


In [16]:
df_wrong_provinces

,address,province,province_key,district_key,ward_key,wrong_province_key
0,Xã Tịnh HàTỉnh Quảng NgãiHuyện Sơn Tịnh,Quảng Ngãi,quangngai,sontinh,tinhha,hatinh
1,Thị trấn Cái Tàu HạĐồng ThápHuyện Châu Thành,Đồng Tháp,dongthap,chauthanh,caitauha,phuyen
2,Xã An HiệpĐồng ThápHuyện Châu Thành,Đồng Tháp,dongthap,chauthanh,anhiep,phuyen
3,Xã An NhơnĐồng ThápHuyện Châu Thành,Đồng Tháp,dongthap,chauthanh,annhon,phuyen
4,Xã Tân Nhuận ĐôngĐồng ThápHuyện Châu Thành,Đồng Tháp,dongthap,chauthanh,tannhuandong,phuyen
...,...,...,...,...,...,...
427,"Vĩnh Mỹ B,Bạc Liêu,Hòa Bình",Bạc Liêu,baclieu,hoabinh,vinhmyb,hoabinh
428,"Vĩnh Hậu,Bạc Liêu,Hòa Bình",Bạc Liêu,baclieu,hoabinh,vinhhau,hoabinh
429,"Vĩnh Hậu A,Bạc Liêu,Hòa Bình",Bạc Liêu,baclieu,hoabinh,vinhhaua,hoabinh
430,"Vĩnh Mỹ A,Bạc Liêu,Hòa Bình",Bạc Liêu,baclieu,hoabinh,vinhmya,hoabinh


In [17]:
DICT_double_check_inverted_provinces = {}
for row in df_wrong_provinces.itertuples():
    tmp_df = df_wrong_provinces[df_wrong_provinces['wrong_province_key']==row.wrong_province_key]
    tmp_province_keys = tmp_df['province_key'].unique().tolist()
    province_data = {}
    for province_key in tmp_province_keys:
        ward_district_keys = tmp_df[tmp_df['province_key']==province_key][['ward_key', 'district_key']].drop_duplicates().values.tolist()
        
        ward_district_keys = sorted(ward_district_keys, key=len, reverse=True)
        province_data[province_key] = ward_district_keys
    
    province_data = dict(sorted(province_data.items(), key=lambda x: len(x[0]), reverse=True))
    DICT_double_check_inverted_provinces[row.wrong_province_key] = province_data

In [18]:
DICT_double_check_inverted_provinces

{'hatinh': {'quangngai': [['tinhha', 'sontinh']],
  'phuyen': [['sonha', 'sonhoa']]},
 'phuyen': {'dongthap': [['caitauha', 'chauthanh'],
   ['anhiep', 'chauthanh'],
   ['annhon', 'chauthanh'],
   ['tannhuandong', 'chauthanh'],
   ['tanbinh', 'chauthanh'],
   ['tanphutrung', 'chauthanh'],
   ['phulong', 'chauthanh'],
   ['anphuthuan', 'chauthanh'],
   ['phuhuu', 'chauthanh'],
   ['ankhanh', 'chauthanh'],
   ['tanphu', 'chauthanh'],
   ['hoatan', 'chauthanh']],
  'sonla': [['phuyen', 'phuyen'],
   ['suoito', 'phuyen'],
   ['muongthai', 'phuyen'],
   ['muongcoi', 'phuyen'],
   ['quanghuy', 'phuyen'],
   ['huybac', 'phuyen'],
   ['huythuong', 'phuyen'],
   ['tanlang', 'phuyen'],
   ['giaphu', 'phuyen'],
   ['tuongphu', 'phuyen'],
   ['huyha', 'phuyen'],
   ['huytan', 'phuyen'],
   ['muonglang', 'phuyen'],
   ['suoibau', 'phuyen'],
   ['huytuong', 'phuyen'],
   ['muongdo', 'phuyen'],
   ['sapxa', 'phuyen'],
   ['tuongthuong', 'phuyen'],
   ['tuongtien', 'phuyen'],
   ['tuongphong', 'phuyen

## Find wards that are the same keyword with districts

We have to add `double_check_provinces` to module to solve province level before doing this step.

In [19]:
wrong_districts = []
for address_col in address_cols:
    for row in df.itertuples():
        address = getattr(row, address_col)

        unit = parse_address(address, level=2)
    
        data = {
            'address': address,
            'province_english': row.province_english,
            'province_key': row.province_key,
            'wrong_province_key': unit.province_key,
            'district_key': row.district_key,
            'ward_key': row.ward_key,
            'wrong_district_key': unit.district_key,
        }
        
                
        if unit.district != row.district:
            print(address)
            wrong_districts.append(data)



Xã Tịnh HàTỉnh Quảng NgãiHuyện Sơn Tịnh
Thị trấn Cái Tàu HạĐồng ThápHuyện Châu Thành
Xã An HiệpĐồng ThápHuyện Châu Thành
Xã An NhơnĐồng ThápHuyện Châu Thành
Xã Tân Nhuận ĐôngĐồng ThápHuyện Châu Thành
Xã Tân BìnhĐồng ThápHuyện Châu Thành
Xã Tân Phú TrungĐồng ThápHuyện Châu Thành
Xã Phú LongĐồng ThápHuyện Châu Thành
Xã An Phú ThuậnĐồng ThápHuyện Châu Thành
Xã Phú HựuĐồng ThápHuyện Châu Thành
Xã An KhánhĐồng ThápHuyện Châu Thành
Xã Tân PhúĐồng ThápHuyện Châu Thành
Xã Hòa TânĐồng ThápHuyện Châu Thành
Xã Tịnh HàTỉnh Quảng NgãiSơn Tịnh
Xã Sơn HàTỉnh Phú YênSơn Hòa
Xã Xuân MinhHà GiangQuang Bình
Xã Tiên NguyênHà GiangQuang Bình
Xã Tân NamHà GiangQuang Bình
Xã Bản RịaHà GiangQuang Bình
Xã Yên ThànhHà GiangQuang Bình
Thị trấn Yên BìnhHà GiangQuang Bình
Xã Tân TrịnhHà GiangQuang Bình
Xã Tân BắcHà GiangQuang Bình
Xã Bằng LangHà GiangQuang Bình
Xã Yên HàHà GiangQuang Bình
Xã Hương SơnHà GiangQuang Bình
Xã Xuân GiangHà GiangQuang Bình
Xã Nà KhươngHà GiangQuang Bình
Xã Tiên YênHà GiangQuang Bình
Xã 

In [20]:
df_wrong_districts = pd.DataFrame(wrong_districts)

In [21]:
df_wrong_districts = df_wrong_districts[df_wrong_districts.province_key==df_wrong_districts.wrong_province_key]

In [22]:
df_wrong_districts

,address,province_english,province_key,wrong_province_key,district_key,ward_key,wrong_district_key


In [23]:
df_wrong_districts[df_wrong_districts.wrong_district_key.isna()]

,address,province_english,province_key,wrong_province_key,district_key,ward_key,wrong_district_key


In [24]:
DICT_double_check_inverted_districts = {}

for province_english in df_wrong_districts['province_english'].unique():
    district_data = {}
    for wrong_district_key in df_wrong_districts[(df_wrong_districts['province_english']==province_english)]['wrong_district_key'].unique():
        tmp_district_keys = df_wrong_districts[(df_wrong_districts['province_english']==province_english) & (df_wrong_districts['wrong_district_key']==wrong_district_key)]['district_key'].unique().tolist()
        district_data[wrong_district_key] = tmp_district_keys
    
    DICT_double_check_inverted_districts[province_english] = district_data

In [25]:
DICT_double_check_inverted_districts

{}

In [26]:
with open('../../vietadminunits/data/parse.pkl', 'rb') as f:
    data = pickle.load(f)

data['DICT_double_check_inverted_provinces'] = DICT_double_check_inverted_provinces
# data['DICT_double_check_inverted_districts'] = DICT_double_check_inverted_districts

with open('../../vietadminunits/data/parse.pkl', 'wb') as f:
    pickle.dump(data, f)